# Rozwiązanie Lab 4

In [1]:
import numpy as np
from scipy import sparse

In [2]:
with open("lines.txt", "r", encoding="UTF-8") as f1:
    unprocessed_e_mails = f1.readlines()

#### Klasa EMAIL
Przechowuje informacje o jednym mailu z pliku lines
self.email - treść maila
self.word_dict - reprezentacja maila bag_of_words, można to utożsamiać z wektorem, lecz jest to dict
self.index_vector - przechowuje id słów, które występują w tym tekście, tej reprezentacji używam przy metrykach dice, lcs, levenshtein
self.vector_norm - norma euklidesowa wektora

In [3]:
class EMAIL:
    def __init__(self, email: str, word_dict: dict[str, int]):
        self.email: str = email
        self.word_dict : dict[str, int] = word_dict
        self.index_vector = None
        self.vector_norm = 1

Processing tekstu

In [4]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from string import punctuation

def process_text(email: str) -> EMAIL:
    text = "".join(list(map(lambda c: " " if c in punctuation else c, email)))
    words = [word.lower() for word in word_tokenize(text)]
    return EMAIL(email, dict(FreqDist(words)))

In [6]:
emails = list(map(process_text, unprocessed_e_mails))

Tworzenie alfabetu (biorę wszystkie słowa jakie występują w tekstach)

In [7]:
alphabet = {}
count = {}
i = 0
for e in emails:
    for word in e.word_dict:
        if word not in alphabet:
            alphabet[word] = i
            count[word] = 1
            i += 1
        else:
            count[word] += 1
M1 = len(alphabet)
N1 = len(emails)
print(M1)
print(N1)

17804
6751


In [8]:
for e in emails:
    indices = list(map(lambda x: alphabet[x], e.word_dict.keys()))
    e.index_vector = np.sort(np.array(indices))

Tworzenie macierzy, w której każda kolumna reprezentuje jakieś słowo jako wektor. Oczywiście jest to macierz rzadka.
Argument words jest to alfabet słów, które występują w mailach w liście mails. M to rozmiar alfabetu a N to ilość maili.
Argument normalize umożliwia stworzenie macierzy w której wektory są znormalizowane (dzięki temu można szybko obliczyć wartości metryki cosinusowej)

In [10]:
def create_sparse_matrix(mails: list[EMAIL], words: dict[str, int], M: int, N: int, normalize=False, idf_values=None):
    nonzero_entries = sum(len(e.index_vector) for e in mails)
    data = np.zeros(nonzero_entries)
    row = np.copy(data)
    col = np.copy(data)
    ind = 0
    for i, e in enumerate(mails):
        start = ind
        for w in e.word_dict:
            data[ind] = e.word_dict[w] * (1 if idf_values is None else idf_values[w])
            row[ind] = words[w]
            col[ind] = i
            ind += 1
        length = np.linalg.norm(data[start:ind])
        e.vector_norm = length
        if normalize:
            data[start:ind] /= length

    return sparse.csr_matrix((data, (row, col)), shape=(M, N), dtype=float)

In [11]:
bag_of_words = create_sparse_matrix(emails, alphabet, M1, N1)
bag_of_words_n = create_sparse_matrix(emails, alphabet, M1, N1, normalize=True)

## Implementacja metryk

### Metryka cosinusowa
Od razu obliczam macierz która zawiera wartość metryki dla każdej pary tekstów. Wyjściowo macierz ma wymiary N x N, gdzie N to liczba tekstów. Obliczanie wartości metryk za każdym razem dla jakiś dwóch teksów zajmowało, bardzo długo, lecz wykorzystując mnożenie macierzy rzadkich, można wartości tych metryk obliczyć bardzo szybko.
Podana macierz musi być już znormalizowana.

In [13]:
def cosine(matrix):
    return np.array((matrix.T @ matrix).todense())

### Metryka euklidesowa
Postępuje analogicznie jak w przypadku metryki cosinusowej i obliczam wartości metryki korzystając z działań na macierzach.

In [14]:
def euclidian(matrix, e_mails, N):
    X = np.array([e.vector_norm**2 for e in e_mails]).reshape((N, 1)) @ np.ones(N).reshape((1, N))
    return np.sqrt(np.abs(X + X.T - 2*np.array((matrix.T @ matrix).todense())))

Dla pozostałych metryk nie da się specjalnie przyśpieszyć obliczania ich wartości. Klasteryzacje z użyciem tych metryk będę przeprowadzał na dużo mniejszym zbiorze tekstów

### Najdłuższy wspólny podciąg

In [15]:
def lcs(text1: EMAIL, text2: EMAIL) -> float:
    seq_a = text1.index_vector
    seq_b = text2.index_vector
    m = len(seq_a)
    n = len(seq_b)
    dp = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq_a[i-1] == seq_b[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    return 1 - (dp[-1][-1] / max(m, n))

### Odległość edycyjna

In [16]:
def levenshtein(text1: EMAIL, text2: EMAIL) -> float:
    seq_a = text1.index_vector
    seq_b = text2.index_vector
    m = len(seq_a)
    n = len(seq_b)
    dp = [[float("inf") for _ in range(n + 1)] for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for i in range(n + 1):
        dp[0][i] = i

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq_a[i-1] == seq_b[j - 1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j-1] + 1, dp[i-1][j] + 1, dp[i][j - 1] + 1)

    return dp[-1][-1] / max(m, n)

### Współczynnik DICE

In [17]:
def dice(text1: EMAIL, text2: EMAIL) -> float:
    seq_a = text1.index_vector
    seq_b = text2.index_vector
    count_union = len([x for x in seq_a if x in seq_b])
    count_1 = len(seq_a)
    count_2 = len(seq_b)
    return 1 - (2*count_union / (count_1 + count_2))

## Klasteryzacja

Do klasteryzacji używam algorytmu k-means. Jest on jednak trochę zmodyfikowany. W oryginalnym algorytmie na początku losowo wybierało się k przedstawicieli grup. Każdy tekst trafiał do grupy do której przedstawiciela miał najbliżej według danej metryki. Następnie kolejnych przedstawicieli się już nie losuje, tylko bierze się średnią z wartości w danym klasterze. Jednak implementacja takiego trudna byłaby trudna, gdyż wtedy pojawiałyby się nowe punkty, a reprezentacja tekstów jako wielowymiarów wektórów, byłaby wymagająca obliczeniowo i prawdopodobnie algorytm nie policzyłby się dla całego zbioru maili. Poza tym pojęcie średniej zmienia zależnie od tego jakiej użyje się metryki. Ponadto przy metryce lcs, dice i levenshtein'a niemożliwe byłoby porównanie tekstów do wektora który powstał ze średniej. Dlatego jako średnią z klustera wybieram po prostu tekst, który znajduje się tym zbiorze i minimalizuje wariancje w obrębie tego klustera. Do oceny klasteryzacji używam indeksu Davies-Bouldine'a

In [21]:
%%time
cosine_metric = cosine(bag_of_words_n)
euclidian_metric = euclidian(bag_of_words, emails, N1)
print(cosine_metric.shape)
print(euclidian_metric.shape)

(6751, 6751)
(6751, 6751)
CPU times: total: 3.97 s
Wall time: 4.28 s


In [22]:
%%time
dice_metric = np.array([[dice(mail1, mail2) for mail1 in emails[:200]] for mail2 in emails[:200]])
lcs_metric = np.array([[lcs(mail1, mail2) for mail1 in emails[:200]] for mail2 in emails[:200]])
edit_metric = np.array([[levenshtein(mail1, mail2) for mail1 in emails[:200]] for mail2 in emails[:200]])

CPU times: total: 18 s
Wall time: 19.1 s


In [25]:
def find_centroid(cluster, metric):
    dists = metric[np.ix_(cluster, cluster)]
    variances = np.power(np.sum(dists, axis=1), 2)
    return cluster[np.argmin(variances)], np.min(variances)


def k_means_cluster(mails, metric=euclidian_metric, max_iter=20, k=10):
    n = metric.shape[0]
    a = np.arange(n)
    np.random.shuffle(a)
    cluster_means = a[:k]
    best_clustering = cluster_means
    best_variance = float("inf")

    for i in range(max_iter):
        clusters = [[cluster_means[i]] for i in range(k)]
        for i, mail in enumerate(mails):
            if i not in cluster_means:
                clusters[np.argmin(metric[i, cluster_means])].append(i)

        cluster_variance = 0
        prev_clustering = cluster_means.copy()

        for i in range(k):
            centroid, min_var = find_centroid(clusters[i], metric)
            cluster_means[i] = centroid
            cluster_variance += min_var

        if cluster_variance < best_variance:
            best_variance = cluster_variance
            best_clustering = cluster_means.copy()

        if np.all(prev_clustering == cluster_means):
            break

    cluster_means = best_clustering
    clusters = [[cluster_means[i]] for i in range(k)]
    for i, mail in enumerate(mails):
        if i not in cluster_means:
            clusters[np.argmin(metric[i, cluster_means])].append(i)

    return clusters


In [26]:
def davies_bouldin_index(clusters, metric):
    k = len(clusters)
    dispersion = []
    centroids = []
    for i in range(k):
        centroid, min_var = find_centroid(clusters[i], metric)
        dispersion.append(min_var / len(clusters[i]))
        centroids.append(centroid)

    separation = metric[np.ix_(centroids, centroids)]
    eps = pow(10, -3)
    separation[np.abs(separation) < eps] = 1000
    np.fill_diagonal(separation, 1000)
    similarity = np.array([[dispersion[i] + dispersion[j] for i in range(k)] for j in range(k)]) / separation

    return np.sum(np.max(similarity, axis=1)) / k

In [27]:
from copy import deepcopy

def find_best_clusters(mails, k_range, metric):
    best = float("inf")
    best_clusters = []
    quality = []
    for i in k_range:
        print(i)
        clusters_ = k_means_cluster(mails, metric=metric, k=i)
        quality.append(davies_bouldin_index(clusters_, metric))
        if quality[-1] < best:
            best = quality[-1]
            best_clusters = deepcopy(clusters_)
    return best_clusters, quality

In [28]:
def print_clusters(clusters):
    for cluster in clusters:
        print("######################")
        for ind in cluster:
            print(emails[ind].email)
    print()

In [29]:
def print_clusters_parameters(clusters, q):
    print(f"Davies-Bouldin index value {q}")
    print(f"Łącznie jest {len(clusters)} grup maili")
    print(f"Największy kluster składa się z {max(len(x) for x in clusters)} maili")

In [ ]:
b_clusters, quality = find_best_clusters(emails, range(200, 3000, 50), euclidian_metric)

## Preprocessing alfabetu

In [211]:
alphabet2 = {}
ind = 0
for word in alphabet:
    if count[word] > 1:
        alphabet2[word] = ind
        ind += 1

In [212]:
def remove_words(mail: EMAIL):
    new_word_dict = {}
    for w in mail.word_dict:
        if w in alphabet2:
            new_word_dict[w] = mail.word_dict[w]
    if len(new_word_dict) == 0:
        return None
    return EMAIL(mail.email, new_word_dict)

In [213]:
emails2 = list(filter(lambda x: x is not None, map(remove_words, emails)))
for e in emails2:
    indices = list(map(lambda x: alphabet2[x], e.word_dict.keys()))
    e.index_vector = np.sort(np.array(indices))

M2 = len(alphabet2)
N2 = len(emails2)
print(M2)
print(N2)

7224
6750


In [214]:
idf = {}
for w in alphabet2:
    idf[w] = np.log(M2 / count[w])

In [217]:
bag_of_words2 = create_sparse_matrix(emails2, alphabet2, M2, N2, idf_values=idf)
bag_of_words_n2 = create_sparse_matrix(emails2, alphabet2, M2, N2, idf_values=idf, normalize=True)

In [227]:
cosine_metric2 = cosine(bag_of_words_n2)
euclidian_metric2 = euclidian(bag_of_words2, emails2, N2)
print(cosine_metric2.shape)
print(euclidian_metric2.shape)

6750
(6750, 6750)
(6750, 6750)


In [ ]:
b_clusters = find_best_clusters(emails2, range(200, 3000, 50), euclidian_metric2)